In [1]:
import numpy as np
from numpy.linalg import svd
from numpy import sqrt
from scipy.optimize import linear_sum_assignment

from arsenal.maths import random_dist
from mdp import DiscountedMDP, MRP, random_MDP, random_MDP_forward_reward
from utils import *

In [2]:
def random_planted_MDP(S, A, count, epsilon, gamma=0.95, b = None):
    
    if b is None: b = S

    phi = np.zeros((S,A))
    for s in range(S):
        phi[s,np.random.randint(A)] = 1
    
    alive = np.arange(S)[:count]
    dead = np.arange(S)[count:]
    
    s0 = np.zeros(S)
    s0[alive] = random_dist(count)
    s0[dead] = epsilon * random_dist(S - count)
    s0 /= np.sum(s0)

    
    P = np.zeros((S,A,S))
    for s in range(S):
        for a in range(A):
            if phi[s,a] == 1:
                connected = np.random.choice(alive, size=min(count, b), replace=False)
                P[s,a,connected] = random_dist(min(count, b))
                
                connected = np.random.choice(dead, size=b, replace=False)
                P[s,a,connected] = epsilon * random_dist(b)
                
                P[s,a,:] /= np.sum(P[s,a,:])
            else:
                connected = np.random.choice(S, size=b, replace=False)
                P[s,a,connected] = random_dist(b)

    R = np.zeros((S,A,S))

    mdp = DiscountedMDP(
        s0 = s0,
        R = R,
        P = P,
        gamma = gamma,
    )

    return mdp, phi

In [3]:
def generate_isomorphic_mdp(mdp):
    S, A, _ = mdp.P.shape
    perm = np.random.permutation(S)
    Pi = np.zeros((S,S))
    I_a = np.eye(A)
    
    for s in range(S):
        Pi[s, perm[s]] = 1
    
    P = np.transpose(mdp.P, (2, 1, 0))
    P = P.reshape((S, A * S))
    P_hat = Pi.T @ P @ np.kron(I_a, Pi)
    P_hat = P_hat.reshape((S, A, S))
    P_hat = np.transpose(P_hat, (2, 1, 0))
    
    mdp_hat = DiscountedMDP(
        s0 = Pi.T @ mdp.s0,
        R = np.zeros((S,A,S)),
        P = P_hat,
        gamma = mdp.gamma,
    )
    
    return mdp_hat, Pi
    

In [4]:
def factor_approximation(M, u, t):
    D = np.diag(u)
    I = np.argwhere(u >= t).flatten()

    M = subindex(M, I)
    D = subindex(D, I)

    _, _, Vh = svd(sqrt(D) @ M @ invsqrt(D))
    V = Vh.T

    colsum = np.sum(V, axis = 0, keepdims = True)
    colsum = np.where(colsum > 0, 1, -1)
    return V * colsum, I

In [5]:
def policy_learning(mdp, phi, t, samples):
    m = len(samples)
    S, A = phi.shape
    Phi = to_policy_matrix(phi)
    M = (mdp | phi).P_with_reset()
    u = (mdp | phi).d()
    
    N = np.zeros((S,S))
    for i in range(m):
        s, sp = samples[i]
        N[s, sp] += 1
    
    u_hat = np.sum(N, axis = 1) / m
    
    denom = np.sum(N, axis = 1, keepdims = True)
    denom[denom == 0] = np.inf
    M_hat = N / denom
#     M_hat = N / np.sum(N, axis = 1, keepdims = True)
#     M_hat = np.nan_to_num(M_hat)
    
    V, I = factor_approximation(M, u, t)
    try:
        V_hat, I_hat = factor_approximation(M_hat, u_hat, t)
    except np.linalg.LinAlgError:
        return None, None
    
    _, perm = linear_sum_assignment(-1 * V @ V_hat.T)
    Pi = np.zeros((S,S))

    I_C = np.setdiff1d(np.arange(S), I)
    I_hat_C = np.setdiff1d(np.arange(S), I_hat)

    dummy_perm = np.random.permutation(I_C.size)
    
    for s in range(S):
        if s in I:
            index = np.where(I == s)[0][0]
            index = perm[index]
            Pi[s, I_hat[index]] = 1
        else:
            index = np.where(I_C == s)[0][0]
            index = dummy_perm[index]
            Pi[s, I_hat_C[index]] = 1
            
            
    Phi_hat = np.kron(np.eye(A), Pi.T) @ Phi @ Pi
    return from_policy_matrix(Phi_hat), Pi
    

In [6]:
#####################################################################

In [7]:
def run_trial(S, A, count, epsilon, b, sample_total = 10000, gamma = 0.95, planted = True):
    if planted:
        mdp, phi = random_planted_MDP(S, A, count, epsilon, gamma, b)
    else:
        mdp = random_MDP(S, A, gamma, b)
        vi = mdp.solve_by_policy_iteration()
        phi = vi['policy']
    
    mdp_hat, Pi_star = generate_isomorphic_mdp(mdp)
    
    Phi = to_policy_matrix(phi)
    Phi_star = np.kron(np.eye(A), Pi_star.T) @ Phi @ Pi_star
    phi_star = from_policy_matrix(Phi_star)
    
    samples = draw_samples(mdp_hat, phi_star, sample_total)
    
    if planted:
        u = np.sort(state_occupancy(mdp, phi))[::-1]
        t = (u[count-1] + u[count]) / 2
        gap = np.abs(u[count-1] - u[count])
    else:
        t = 0.0
        gap = -1

    phi_hat, Pi = policy_learning(mdp, phi, t, samples)
    if phi_hat is None:
        return -999, -999

    d = full_occupancy(mdp, phi)
    d_hat = full_occupancy(mdp_hat, phi_hat)

    return tv(np.kron(np.eye(A), Pi_star.T) @ d.T.flatten(), d_hat.T.flatten()), gap

In [12]:
def run_trials(trials, S, A, count, epsilon, b, sample_total = 10000, gamma = 0.95, planted = True):
    tvs = []
    gaps = []
    for _ in range(trials):
        diff, gap = run_trial(S, A, count, epsilon, b, sample_total, gamma, planted)
        tvs.append(diff)
        gaps.append(gap)
    tvs = np.array(tvs)
    gaps = np.array(gaps)
    return np.mean(tvs), np.std(tvs)

In [13]:
#####################################################################

In [21]:
trials = 30
A = 5
b = 5
sample_total = 50000
gamma = 0.95

In [22]:
#Planted runs

count = 10
epsilon = 0.0001
planted = True

for S in [20, 40, 60, 80, 100]:
    print(run_trials(trials, S, A, count, epsilon, b, sample_total, gamma, planted))

(0.07225494386562355, 0.15248898128082403)
(0.0645101584506826, 0.1649580473900227)


KeyboardInterrupt: 

In [17]:
#Unplanted runs

count = 0
epsilon = 0
planted = False

for S in [20, 40, 60, 80, 100]:
    print(run_trials(trials, S, A, count, epsilon, b, sample_total, gamma, planted))

(0.25270576391401484, 0.2254255853524616)
(0.422669617392449, 0.19781312116249314)


/Users/aaronzweig/Projects/third-person/utils.py:75: RuntimeWarning: divide by zero encountered in true_divide
  d = 1.0 / sqrt(d)
/anaconda3/envs/prime/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in matmul
  


(-199.36764814584612, 399.8162011944866)
(-499.17021430890816, 499.8297902179589)
(-399.1898214589917, 489.74296441607055)
